In [ ]:
import boto3
from datetime import datetime

# Print execution info
print(f"Execution started at: {datetime.now()}")

# Get current IAM role
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"Current Role ARN: {identity['Arn']}")
print(f"Account ID: {identity['Account']}")

In [ ]:
import os
print(f"IMAGE_VERSION: {os.environ.get('IMAGE_VERSION', 'Not set')}")
print(f"SAGEMAKER_INTERNAL_IMAGE_URI: {os.environ.get('SAGEMAKER_INTERNAL_IMAGE_URI', 'Not set')}")

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
print(f"Execution Start Time (EST): {datetime.now(ZoneInfo('America/New_York')).strftime('%Y-%m-%d %H:%M:%S %Z')}")

In [ ]:
# Package upgrade skipped - using environment default
print('Using default sagemaker_studio package from environment')

# Test 10: Time Series Forecasting with Chronos

This notebook demonstrates how to use SageMaker Python SDK to deploy a SageMaker JumpStart time series forecasting model and invoke the endpoint

In [0]:
model_id = "autogluon-forecasting-chronos-t5-small"
model_version = "*"

## Deploy model
Create a JumpStartModel object, which initializes default model configurations conditioned on the selected instance type. JumpStart already sets a default instance type, but you can deploy the model on other instance types by passing instance_type to the JumpStartModel class.

In [0]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, model_version=model_version)
model

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.xlarge.


2025-11-02 17:08:04,716 - sagemaker.jumpstart - INFO - No instance type selected for inference hosting endpoint. Defaulting to ml.g5.xlarge.


You can now deploy the model using SageMaker JumpStart. The deployment might take a few minutes.

In [0]:
from pprint import pformat

def nested_round(data, decimals=3):
    """Round numbers, including nested dicts and list."""
    if isinstance(data, float):
        return round(data, decimals)
    elif isinstance(data, list):
        return [nested_round(item, decimals) for item in data]
    elif isinstance(data, dict):
        return {key: nested_round(value, decimals) for key, value in data.items()}
    else:
        return data


def pretty_format(data):
    return pformat(nested_round(data), width=150, sort_dicts=False)

In [0]:
for payload in model.retrieve_all_examples():
    response = predictor.predict(payload.body)
    print("Input:\n", pretty_format(payload.body), end="\n\n")
    print("Output:\n", pretty_format(response))
    print("\n===============\n")

Input:
 {'inputs': [{'target': [0.0, 4.0, 5.0, 1.5, -3.0, -5.0, -3.0, 1.5, 5.0, 4.0, 0.0, -4.0, -5.0, -1.5, 3.0, 5.0, 3.0, -1.5, -5.0, -4.0]}],
 'parameters': {'prediction_length': 10}}

Output:
 {'predictions': [{'mean': [0.231, 3.432, 3.755, 2.038, -0.662, -2.835, -2.55, 0.419, 3.038, 2.493],
                  '0.1': [-3.012, 1.501, 1.485, 0.0, -3.092, -5.117, -5.18, -2.982, 0.676, 0.0],
                  '0.5': [1.501, 4.012, 4.012, 1.947, -0.809, -4.95, -3.003, 1.501, 3.003, 3.003],
                  '0.9': [3.003, 5.128, 5.098, 4.012, 3.202, 1.501, 0.826, 3.003, 4.997, 4.997]}]}


Input:
 {'inputs': [{'target': [1.0, 2.0, 3.0, 2.0, 0.5, 2.0, 3.0, 2.0, 1.0], 'item_id': 'product_A', 'start': '2024-01-01T01:00:00'},
            {'target': [5.4, 3.0, 3.0, 2.0, 1.5, 2.0, -1.0], 'item_id': 'product_B', 'start': '2024-02-02T03:00:00'}],
 'parameters': {'prediction_length': 5, 'freq': '1h', 'quantile_levels': [0.05, 0.5, 0.95], 'num_samples': 30, 'batch_size': 2}}

Output:
 {'predictions'